In [5]:
path =  r"D:\Data\NYC\KINZ\KINECT_ACC_dataset_with_qor15_2025-05-27_14-29PM.csv"
dataframe = pd.read_csv(path)

C:\Users\user\AppData\Local\Temp\ipykernel_15116\4173027433.py:2: DtypeWarning: Columns (108,112,113,117,151,154,161,162,164,169,180,185,198,221,228,264,267,269,270,306,307,310,311,314,315,318,319,322,323,326,328,330,331,334,335,338,339,342,343,346,348,350,352,354,356,358,362,363,393,403,404,406,418,423,430,433,436,439,444) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(path)


In [6]:
dataframe

,patientID,trial_num,walking_speed,t_uniform,PELVIS_X,SPINE_NAVAL_X,SPINE_CHEST_X,NECK_X,CLAVICLE_LEFT_X,SHOULDER_LEFT_X,...,comp4_type,comp4_cd,comp4_desc,comp5_type,comp5_cd,comp5_desc,cc_index,notes,postoperative_3day_data_complete,QoR_class
0,001-LO,Trial4,Fast,0.001646,-36.173563,-32.222103,-29.139376,-24.572845,6.011915,125.887829,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2.0,1.0
1,001-LO,Trial4,Fast,0.021627,-26.631417,-24.712284,-23.071704,-19.095052,11.770258,132.878366,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2.0,1.0
2,001-LO,Trial4,Fast,0.041607,-17.089270,-17.202465,-17.004033,-13.617259,17.528602,139.868903,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2.0,1.0
3,001-LO,Trial4,Fast,0.061588,-7.547124,-9.692646,-10.936362,-8.139466,23.286945,146.859440,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2.0,1.0
4,001-LO,Trial4,Fast,0.081568,-13.797027,-19.793992,-23.898339,-14.030918,16.551726,142.158210,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116578,104-AB,Trial2,Regular,4.830416,-106.625566,-99.957650,-94.666152,-85.579490,-57.789959,56.025031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
116579,104-AB,Trial2,Regular,4.850396,-102.713773,-95.755603,-90.207959,-81.320193,-53.332484,61.004254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
116580,104-AB,Trial2,Regular,4.870377,-99.019364,-91.877139,-86.144402,-77.671170,-49.452012,65.611329,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
116581,104-AB,Trial2,Regular,4.890357,-95.402811,-88.318498,-82.601346,-74.990126,-46.483008,69.709487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import WeightedRandomSampler
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pickle
from sklearn.model_selection import GroupShuffleSplit


# 1. Load Data
path =  r"D:\Data\NYC\KINZ\KINECT_ACC_dataset_with_qor15_2025-05-27_14-29PM.csv"
dataframe = pd.read_csv(path)
dataframe = dataframe[dataframe['walking_speed'] == 'Fast']


# 2. Graph Definition
joints = ['PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
          'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
          'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
          'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
          'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
          'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']

edges = [('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
         ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
         ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
         ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
         ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
         ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
         ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
         ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
         ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
         ('HEAD', 'NOSE'), ('HEAD','EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')]

joint_to_idx = {joint: idx for idx, joint in enumerate(joints)}
edge_index = torch.tensor([[joint_to_idx[src], joint_to_idx[dst]] for src, dst in edges] +
                          [[joint_to_idx[dst], joint_to_idx[src]] for src, dst in edges],
                          dtype=torch.long).t()

# 3. Dataset
class SkeletonDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.data = df
        self.num_nodes = len(joints)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        node_features = [[row[f'{j}_X'], row[f'{j}_Y'], row[f'{j}_Z'], row['t_uniform']] for j in joints]
        x = torch.tensor(node_features, dtype=torch.float)
        y = torch.tensor(row['QoR_class'], dtype=torch.float)
        return Data(x=x, edge_index=edge_index, y=y)

# 4. Random split
full_dataset = SkeletonDataset(dataframe.dropna(subset=['QoR_class']))
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# 5. Weighted Sampling
train_indices = train_dataset.indices if isinstance(train_dataset, torch.utils.data.Subset) else list(range(len(train_dataset)))
train_df = dataframe.iloc[train_indices].reset_index(drop=True)
train_df = train_df.dropna(subset=['QoR_class'])
class_counts = train_df['QoR_class'].value_counts().sort_index().to_numpy()
class_weights = 1. / class_counts
sample_weights = [class_weights[int(label)] for label in train_df['QoR_class']]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# 6. Dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# 7. Model
class HybridSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super().__init__()
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)
        self.temporal_conv1 = nn.Conv1d(hidden_channels, hidden_channels, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = getattr(data, 'batch', None)
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))
        batch_size = batch.max().item() + 1 if batch is not None else 1
        num_nodes = x.size(0) // batch_size
        x = x.view(batch_size, num_nodes, -1).permute(0, 2, 1)
        x = F.relu(self.temporal_conv1(x)).permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x.mean(dim=1)
        return self.fc(x).squeeze()

# 8. Training Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridSTGCN(in_channels=4, hidden_channels=128, num_classes=1).to(device)
pos_weight = torch.tensor([class_weights[0] / class_weights[1]]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                if self.best_model is not None:
                    model.load_state_dict(self.best_model)
                return True
        return False

# 9. Training Loop
num_epochs = 100
train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
early_stopping = EarlyStopping()

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        y = data.y.to(device).float()
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
        correct += ((out >= 0.5) == y).sum().item()
        total += data.num_graphs
    train_loss = total_loss / len(train_loader.dataset)
    train_acc = correct / total

    model.eval()
    total_loss, correct, total, all_preds, all_labels = 0, 0, 0, [], []
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            y = data.y.to(device).float()
            loss = criterion(out, y)
            total_loss += loss.item() * data.num_graphs
            pred = (torch.sigmoid(out) >= 0.5).float()
            correct += (pred == y).sum().item()
            total += data.num_graphs
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    test_loss = total_loss / len(test_loader.dataset)
    test_acc = correct / total
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}")

    if early_stopping(test_loss, model):
        print("Early stopping triggered.")
        break

# 10. Final Evaluation
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
roc_auc = roc_auc_score(all_labels, all_preds)
tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
specificity = tn / (tn + fp)

print(f"Final Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, target_names=['Good (0)', 'Not Good (1)']))

# 11. Save Model + Stats
now = datetime.now().strftime("%Y%m%d_%H%M%S")
torch.save(model.state_dict(), f"QOR_hybrid_stgcn_model_{now}.pth")
with open(f"QOR_hybrid_stgcn_model_statistics_{now}.pkl", 'wb') as f:
    pickle.dump({
        'train_losses': train_losses,
        'test_losses': test_losses,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies,
        'final_metrics': {
            'accuracy': accuracy,
            'f1_score': f1,
            'recall': recall,
            'precision': precision,
            'specificity': specificity,
            'roc_auc': roc_auc
        }
    }, f)
print("Model and statistics saved.")


C:\Users\user\AppData\Local\Temp\ipykernel_15116\5513061.py:18: DtypeWarning: Columns (108,112,113,117,151,154,161,162,164,169,180,185,198,221,228,264,267,269,270,306,307,310,311,314,315,318,319,322,323,326,328,330,331,334,335,338,339,342,343,346,348,350,352,354,356,358,362,363,393,403,404,406,418,423,430,433,436,439,444) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(path)


Epoch 001 | Train Loss: 0.5386, Acc: 0.5900 | Test Loss: 0.5647, Acc: 0.6210
Epoch 002 | Train Loss: 0.5145, Acc: 0.6129 | Test Loss: 0.5466, Acc: 0.6375
Epoch 003 | Train Loss: 0.4938, Acc: 0.6185 | Test Loss: 0.5235, Acc: 0.6838
Epoch 004 | Train Loss: 0.4741, Acc: 0.6310 | Test Loss: 0.5155, Acc: 0.6908
Epoch 005 | Train Loss: 0.4539, Acc: 0.6546 | Test Loss: 0.4933, Acc: 0.7142
Epoch 006 | Train Loss: 0.4326, Acc: 0.6854 | Test Loss: 0.4609, Acc: 0.7302
Epoch 007 | Train Loss: 0.4109, Acc: 0.7078 | Test Loss: 0.4478, Acc: 0.7504
Epoch 008 | Train Loss: 0.3904, Acc: 0.7293 | Test Loss: 0.4205, Acc: 0.7773
Epoch 009 | Train Loss: 0.3683, Acc: 0.7506 | Test Loss: 0.3935, Acc: 0.7970
Epoch 010 | Train Loss: 0.3442, Acc: 0.7746 | Test Loss: 0.3651, Acc: 0.8126
Epoch 011 | Train Loss: 0.3222, Acc: 0.7923 | Test Loss: 0.3414, Acc: 0.8370
Epoch 012 | Train Loss: 0.3033, Acc: 0.8085 | Test Loss: 0.3338, Acc: 0.8335
Epoch 013 | Train Loss: 0.2868, Acc: 0.8207 | Test Loss: 0.3240, Acc: 0.8313

KeyboardInterrupt: 

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Load dataset
#dataframe = pd.read_csv(r'C:\Users\RAZER\Desktop\KINZ AGAIN\Kinect_feature_matrix_20250307_123820.csv')

# Prepare data
# Include 'timeStamps' as an additional feature
X = dataframe.drop(columns=['QoR_class', 'patientID', 'trial_num', 'walking_speed'])

# Concatenate 'timeStamps' as an additional column to the X dataset
X = pd.concat([X, dataframe['timeStamps']], axis=1)

y = dataframe['QoR_class']
groups = dataframe['patientID']

# Normalize data (standardization)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into train and test using GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in gss.split(X, y, groups):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# CNN-LSTM Hybrid Model Definition
class CNNLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3, dropout=0.5):
        super(CNNLSTMModel, self).__init__()
        
        # Initialize hidden_size as an attribute of the model
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # CNN Layers for feature extraction
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # LSTM Layer
        self.lstm = nn.LSTM(256, hidden_size, num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size * 2, output_size)  # *2 for bidirectional
        
        # Batch Normalization
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        
    def forward(self, x):
        # Apply CNN layers with Batch Normalization
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        
        # Prepare data for LSTM (must reshape to [batch_size, seq_len, input_size])
        x = x.permute(0, 2, 1)  # Change shape to [batch_size, seq_len, input_size]
        
        # LSTM forward pass
        h0 = torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 2 for bidirectional
        c0 = torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        # Use the last hidden state for classification
        out = out[:, -1, :]
        
        # Fully connected layer
        out = self.fc(out)
        return out

# Model hyperparameters
input_size = X_train.shape[1]  # Now includes 'timeStamps', so input_size should be 102
hidden_size = 128  # Increased hidden size to capture more complex patterns
output_size = len(y.unique())  # Number of classes
model = CNNLSTMModel(input_size=1, hidden_size=hidden_size, output_size=output_size, num_layers=3, dropout=0.5)  # Adjusted input_size for CNN

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.001)  # Lower learning rate and higher weight decay for L2 regularization

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Early stopping setup
patience = 5
best_test_acc = 0
epochs_without_improvement = 0

# Training Loop
epochs = 100
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_train, total_train = 0, 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        # Reshape the data to [batch_size, num_channels, sequence_length]
        X_batch = X_batch.unsqueeze(1)  # Adding channel dimension (1 for single feature per time step)
        
        outputs = model(X_batch)  # Pass through CNN-LSTM model
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Compute training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += y_batch.size(0)
        correct_train += (predicted == y_batch).sum().item()

    avg_train_loss = total_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_acc)

    # Evaluate on test set
    model.eval()
    test_loss = 0
    correct_test, total_test = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Reshape the data to [batch_size, num_channels, sequence_length]
            X_batch = X_batch.unsqueeze(1)  # Adding channel dimension (1 for single feature per time step)
            
            outputs = model(X_batch)  # Pass through CNN-LSTM model
            loss = criterion(outputs, y_batch)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_test += y_batch.size(0)
            correct_test += (predicted == y_batch).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    test_acc = 100 * correct_test / total_test
    test_losses.append(avg_test_loss)
    test_accuracies.append(test_acc)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%")

    # Early stopping check
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping triggered.")
            break

    scheduler.step()

# Save the model after training
torch.save(model.state_dict(), 'QOR_classification_cnn_lstm.pth')
print("CNN-LSTM Model saved successfully!")

# Final Test Accuracy
print(f'Final Test Accuracy: {test_accuracies[-1]:.2f}%')

# Confusion Matrix
all_true_labels = []
all_pred_labels = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Reshape the data to [batch_size, num_channels, sequence_length]
        X_batch = X_batch.unsqueeze(1)  # Adding channel dimension (1 for single feature per time step)
        
        outputs = model(X_batch)  # Pass through CNN-LSTM model
        _, predicted = torch.max(outputs, 1)
        all_true_labels.extend(y_batch.cpu().numpy())
        all_pred_labels.extend(predicted.cpu().numpy())



KeyError: 'timeStamps'